In [ ]:
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d shivasai1909/isl-final-split-dataset
!unzip isl-final-split-dataset

In [ ]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
import os

In [ ]:
train_dir = '/content/final_split_dataset/train'
val_dir = '/content/final_split_dataset/val'

In [ ]:
img_width, img_height = 128, 128

In [ ]:
# Data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Rescaling for validation set
val_datagen = ImageDataGenerator(rescale=1./255)

# Training data generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    class_mode='categorical',
    color_mode='grayscale'
)

# Validation data generator
validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    class_mode='categorical',
    color_mode='grayscale'
)

Found 55424 images belonging to 27 classes.
Found 13869 images belonging to 27 classes.


In [ ]:
class_names = list(train_generator.class_indices.keys())
dataset_dir = '/content/final_split_dataset'
checkpoint_dir = os.path.join('/content/final_split_dataset', '.ipynb_checkpoints')

# Check if the .ipynb_checkpoints directory exists
if os.path.exists(checkpoint_dir) and os.path.isdir(checkpoint_dir):
    # If it exists, remove it
    os.rmdir(checkpoint_dir)
print(class_names)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'blank']


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from keras.callbacks import EarlyStopping, TensorBoard
import os

In [ ]:
model = Sequential()

# Convolutional layers
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(img_width, img_height, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

# Flatten layer
model.add(Flatten())

# Fully connected layers
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Output layer
model.add(Dense(27, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 64)      640       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 64)        0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 63, 63, 64)        256       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 63, 63, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 128)       0

In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy' )

In [ ]:
logdir = os.path.join("Logs")
tensorboard_callback = TensorBoard(log_dir=logdir)


In [ ]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[tensorboard_callback, early_stopping_callback]
)

Epoch 1/20
   9/1732 [..............................] - ETA: 1:55:45 - loss: 4.5378 - accuracy: 0.0590

In [ ]:
%load_ext tensorboard
%tensorboard --logdir Logs

In [ ]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print("Validation Accuracy:", val_accuracy)
print("Validation Loss:", val_loss)

In [ ]:
model_json = model.to_json()
with open("signlanguageb128.json",'w') as json_file:
    json_file.write(model_json)
model.save("signlanguageb128.h5")